In [1]:
import os

In [2]:
BUCKET = 'gs://edml/data/taxi-trips'

In [3]:
os.environ['BUCKET'] = BUCKET

## Local training / Python

In [4]:
%%bash
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer
echo $PYTHONPATH

:/home/jupyter/event-driven-ml/edml-trainer


In [5]:
%%bash
echo "bucket=${BUCKET}"
rm -rf model_trained_test
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer
python3 -m trainer.task \
  --bucket=${BUCKET} \
  --output-dir=model_trained_test \
  --job-dir=./tmp \
  --train-examples=500 \
  --nembeds 10 \
  --nnsize 10 5 \
  --eval-steps=2

bucket=gs://edml/data/taxi-trips


INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_global_id_in_cluster': 0, '_is_chief': True, '_keep_checkpoint_max': 3, '_protocol': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc38ee2d080>, '_num_worker_replicas': 1, '_task_type': 'worker', '_eval_distribute': None, '_tf_random_seed': 2810, '_session_creation_timeout_secs': 7200, '_num_ps_replicas': 0, '_model_dir': 'model_trained_test', '_task_id': 0, '_train_distribute': None, '_save_summary_steps': 100, '_experimental_max_worker_delay_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_service': None, '_experimental_distribute': None, '_device_fn': None, '_save_checkpoints_steps': None, '_master': '', '_save_checkpoints_secs': 300, '_log_step_count_steps': 100}
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, 

In [ ]:
%%bash
gsutil cp ./model_trained_test/export/exporter/1581540201/saved_model.pb gs://edml/tmp

In [ ]:
from google.datalab.ml import TensorBoard

In [ ]:
TensorBoard().start('./model_trained_test')
TensorBoard().list()

In [ ]:
for pid in TensorBoard.list()['pid']:
    TensorBoard().stop(pid)
    print('Stopped TensorBoard with pid {}'.format(pid))

In [ ]:
%%bash
export MODEL_DIR=tmp_tensorboard
tensorboard --logdir=$MODEL_DIR --port=6006

## Local prediction

In [6]:
%%writefile inputs.json
{"uuid": "b1", "dayofweek": 6, "hourofday": 7, "pickup_zone_name": "World Trade Center", "dropoff_zone_name": "Newark Airport", "passenger_count": 1}
{"uuid": "g1", "dayofweek": 3, "hourofday": 23, "pickup_zone_name": "World Trade Center", "dropoff_zone_name": "Times Sq/Theatre District", "passenger_count": 1}

Overwriting inputs.json


In [7]:
%%bash
MODEL_LOCATION=$(ls -d $(pwd)/model_trained_test/export/exporter/* | tail -1)
echo $MODEL_LOCATION
gcloud ai-platform local predict --model-dir=$MODEL_LOCATION --json-instances=inputs.json

/home/jupyter/event-driven-ml/edml-notebooks/gbi/model_trained_test/export/exporter/1582746870
PREDICTIONS
[6.1988983154296875]
[4.54083251953125]


If the signature defined in the model is not serving_default then you must specify it via --signature-name flag, otherwise the command may fail.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





2020-02-26 19:56:20.364659: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-02-26 19:56:20.366094: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5569d06b2390 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-02-26 19:56:20.366128: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-02-26 19:56:20.366529: I tensorflow/core/co

## Local training / Gcloud

In [12]:
%%bash
gcloud ai-platform --help

NAME
    gcloud ai-platform - manage AI Platform jobs and models

SYNOPSIS
    gcloud ai-platform GROUP | COMMAND [GCLOUD_WIDE_FLAG ...]

DESCRIPTION
    The gcloud ai-platform command group lets you manage AI Platform jobs and
    training models.

    AI Platform is a managed service that enables you to easily build machine
    learning models, that work on any type of data, of any size. Create your
    model with the powerful TensorFlow framework that powers many Google
    products, from Google Photos to Google Cloud Speech.

    More information on AI Platform can be found here:
    https://cloud.google.com/ml and detailed documentation can be found here:
    https://cloud.google.com/ml/docs/

GCLOUD WIDE FLAGS
    These flags are available to all commands: --account, --billing-project,
    --configuration, --flags-file, --flatten, --format, --help,
    --impersonate-service-account, --log-http, --project, --quiet,
    --trace-token, --user-output-enabled, --verbosity.

    Run $ 

In [6]:
%%bash
echo $PATH

/usr/local/cuda/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


In [7]:
%%bash
rm -rf model_trained_test
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer:/usr/bin/python
gcloud ai-platform local train \
   --module-name=trainer.task \
   --package-path=~/event-driven-ml/edml-trainer \
   --job-dir=./tmp \
   -- \
   --train-examples=500 \
   --output-dir=model_trained_test \
   --eval-steps=1

ERROR: (gcloud.ai-platform.local.train) /usr/bin/python: command not found


CalledProcessError: Command 'b'rm -rf model_trained_test\nexport PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer\ngcloud ai-platform local train \\\n   --module-name=trainer.task \\\n   --package-path=~/event-driven-ml/edml-trainer \\\n   --job-dir=./tmp \\\n   -- \\\n   --train-examples=500 \\\n   --output-dir=model_trained_test \\\n   --eval-steps=1\n'' returned non-zero exit status 1